In [1]:
using BenchmarkTools

In [2]:
function rotate(matrix)
    return reverse(reverse(matrix, dims=1), dims=2)
end

rotate (generic function with 1 method)

In [3]:
function Convolution_2d_pad(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d_pad (generic function with 1 method)

In [4]:
function Convolution_2d(input, kernel; bias=0.)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [5]:
function Convolution_2d!(ret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    sumret = zeros(size(kernel))
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

function Convolution_2d_v2!(ret, sumret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

Convolution_2d_v2! (generic function with 1 method)

In [6]:
function Conv_backward_v1( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_input[:, :, k] += Convolution_2d_pad(weights[:, :, k, c], gradient[:, :, c]; padding=true)
        end
    end

    grad_weights = zeros(Float32, size(weights))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_weights[:, :, k, c] += Convolution_2d(input[:, :, k], gradient[:, :, c])
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v1 (generic function with 1 method)

In [7]:
function Conv_backward_v2(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            grad_input[:, :, k] .+= Convolution_2d_pad(tmp_weights, rotate(tmp_gradient); padding=true)
        end
    end

    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v2 (generic function with 1 method)

In [8]:
function Conv_backward_v3( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v3 (generic function with 1 method)

In [9]:
function Conv_backward_v4(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
            grad_weights[:, :, k, c] .+= tmp_weights
            tmp_weights .= 0
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v4 (generic function with 1 method)

In [10]:
function Conv_backward_v5(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  tmp_weight_gradient = zeros(Float32, kernel_height, kernel_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
          grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v5 (generic function with 1 method)

In [11]:
function Conv_backward_v6(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  tmp_weight_gradient = zeros(Float32, kernel_height, kernel_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
          @views grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v6 (generic function with 1 method)

In [12]:
function Conv_backward_v7(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  sumret = zeros(Float32, output_height, output_width)
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d_v2!(tmp_weights, sumret, tmp_input, tmp_gradient)
          @views grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v7 (generic function with 1 method)

In [13]:
input = rand(28, 28, 1);
weights = rand(3, 3, 1, 6);
bias = rand(Float32, 6);
gradient = rand(26, 26, 6);

In [14]:
@benchmark Conv_backward_v1(input, weights, bias, gradient)

BenchmarkTools.Trial: 1142 samples with 1 evaluation.
 Range (min … max):  2.921 ms … 17.388 ms  ┊ GC (min … max):  0.00% … 38.15%
 Time  (median):     4.040 ms              ┊ GC (median):    25.61%
 Time  (mean ± σ):   4.366 ms ±  2.300 ms  ┊ GC (mean ± σ):  18.26% ± 13.78%

  █▄▃▁▁▇▆▅▄▁                                                  
  ███████████▆▅▁▄▁▄▅▄▁▁▁▁▁▁▄▅▁▅▁▅▄▁▁▁▅▁▁▇▅▅▇▆▇▁▄▅▅▄▅▁▄▅▄▆▁▅▆ █
  2.92 ms      Histogram: log(frequency) by time     14.9 ms <

 Memory estimate: 25.49 MiB, allocs estimate: 4842.

In [15]:
@benchmark Conv_backward_v2(input, weights, bias, gradient)

BenchmarkTools.Trial: 1283 samples with 1 evaluation.
 Range (min … max):  2.751 ms … 16.973 ms  ┊ GC (min … max):  0.00% … 61.78%
 Time  (median):     3.086 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.889 ms ±  1.413 ms  ┊ GC (mean ± σ):  15.97% ± 17.71%

  ▄█▄▃▃▁ ▁        ▂▄▃▃▂▁▁▁                                    
  ████████▇▇▇▆▆▇▇█████████▇█▇▆▆▇▆▅▆▇▅▆▆▅▄▁▅▆▆▅▁▅▄▄▄▄▄▄▄▄▄▁▁▄ █
  2.75 ms      Histogram: log(frequency) by time     9.15 ms <

 Memory estimate: 13.22 MiB, allocs estimate: 4827.

In [16]:
@benchmark Conv_backward_v3(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  306.400 μs …   2.386 ms  ┊ GC (min … max): 0.00% … 75.80%
 Time  (median):     316.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   339.189 μs ± 120.120 μs  ┊ GC (mean ± σ):  3.42% ±  7.96%

  █▅▃▁ ▁                                                        ▁
  ████████▇▇█▇▇▇▇▆▇▇▇▇▇▇▆▆▅▅▄▁▁▃▃▄▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▇ █
  306 μs        Histogram: log(frequency) by time       1.23 ms <

 Memory estimate: 544.08 KiB, allocs estimate: 4149.

In [17]:
@benchmark Conv_backward_v4(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  306.300 μs …  1.555 ms  ┊ GC (min … max): 0.00% … 73.74%
 Time  (median):     312.500 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   328.111 μs ± 72.787 μs  ┊ GC (mean ± σ):  1.68% ±  6.02%

  ██▃▃▂▂▁                                                      ▁
  ████████▇▆▇▆▆▆▆▆▅▆▅▆▆▆▅▆▄▆▇▄▆▅▅▅▆▅▅▅▄▄▅▅▃▄▄▄▆▅▂▄▂▅▅▅▅▃▃▅▄▂▃▃ █
  306 μs        Histogram: log(frequency) by time       660 μs <

 Memory estimate: 423.42 KiB, allocs estimate: 4083.

In [18]:
@benchmark Conv_backward_v5(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  126.800 μs …  2.411 ms  ┊ GC (min … max): 0.00% … 88.23%
 Time  (median):     132.500 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   154.474 μs ± 71.802 μs  ┊ GC (mean ± σ):  0.98% ±  2.70%

  ██▇▅▄▄▃▂▂▂▁▁▁               ▁▁▂▂▂▁▂▁▁▁▁▁▁▁                   ▂
  ███████████████▇█▇▇▇█▇▇▇▇▇▇████████████████████▆▇▇▅▆▆▆▅▅▅▅▅▅ █
  127 μs        Histogram: log(frequency) by time       301 μs <

 Memory estimate: 43.27 KiB, allocs estimate: 28.

In [19]:
@benchmark Conv_backward_v6(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  127.500 μs …  1.800 ms  ┊ GC (min … max): 0.00% … 91.03%
 Time  (median):     130.700 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   136.178 μs ± 48.539 μs  ┊ GC (mean ± σ):  0.97% ±  2.71%

  █▇▇▅▅▄▃▂▂▂▁▁▁                                                ▂
  ██████████████▇█▇▇▇▆▆▆▆▆▆▅▅▅▅▆▅▄▄▄▅▄▅▅▄▄▆▄▅▄▄▄▄▅▄▄▅▄▆▅▆▄▅▆▄▄ █
  128 μs        Histogram: log(frequency) by time       226 μs <

 Memory estimate: 42.70 KiB, allocs estimate: 22.

In [20]:
@benchmark Conv_backward_v7(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  129.600 μs …  2.407 ms  ┊ GC (min … max): 0.00% … 92.93%
 Time  (median):     132.400 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   134.176 μs ± 37.438 μs  ┊ GC (mean ± σ):  0.46% ±  1.60%

  ▆█▆▄▃▆▇▆▅▄▃▃▂▃▂▁▁▁   ▁▁                                      ▂
  ███████████████████████▇█▇▇▇▇▆▅▇▇▆▇▇▆▆▇▇▆▆▄▄▅▆▄▄▅▅▅▅▅▅▃▄▄▅▃▅ █
  130 μs        Histogram: log(frequency) by time       159 μs <

 Memory estimate: 13.64 KiB, allocs estimate: 28.

In [21]:
Conv_backward_v4(input, weights, bias, gradient)

(Float32[2.4740026 2.6737921 … 2.2692873 1.7888535; 3.1967802 6.0427833 … 5.377192 2.864847; … ; 2.7728949 5.810988 … 6.598092 2.5808516; 1.2643493 3.2989457 … 3.5650449 1.032854;;;], Float32[167.64406 169.15935 167.00087; 167.70038 163.98868 163.67014; 162.34267 163.4832 165.82878;;;; 163.57365 159.92496 164.4841; 161.50903 160.84991 161.48358; 161.07101 162.28337 162.46985;;;; 167.42563 168.78995 171.7227; 169.2116 166.07845 168.47707; 163.76364 163.80411 171.1175;;;; 176.75003 176.45343 177.18245; 170.60622 175.03868 177.14157; 171.02507 176.13356 177.88799;;;; 163.75743 166.89166 163.69728; 166.04156 163.56065 163.95918; 161.5332 162.68594 158.96266;;;; 171.92052 174.48372 173.80414; 171.91307 176.1706 172.15506; 170.91565 172.85698 175.36737], [338.2929296134087, 328.8824437877334, 341.81983651236527, 355.49778988061513, 333.1912336044469, 352.39460710117095])

In [22]:
Conv_backward_v6(input, weights, bias, gradient)

(Float32[2.4740026 2.673792 … 2.269287 1.7888533; 3.1967802 6.0427833 … 5.377192 2.864847; … ; 2.7728946 5.810988 … 6.5980916 2.5808516; 1.2643493 3.2989457 … 3.5650446 1.032854;;;], Float32[167.64406 169.15935 167.00087; 167.70038 163.98868 163.67014; 162.34267 163.4832 165.82878;;;; 163.57365 159.92496 164.4841; 161.50903 160.84991 161.48358; 161.07101 162.28337 162.46985;;;; 167.42563 168.78995 171.7227; 169.2116 166.07845 168.47707; 163.76364 163.80411 171.1175;;;; 176.75003 176.45343 177.18245; 170.60622 175.03868 177.14157; 171.02507 176.13356 177.88799;;;; 163.75743 166.89166 163.69728; 166.04156 163.56065 163.95918; 161.5332 162.68594 158.96266;;;; 171.92052 174.48372 173.80414; 171.91307 176.1706 172.15506; 170.91565 172.85698 175.36737], [338.2929296134087, 328.8824437877334, 341.81983651236527, 355.49778988061513, 333.1912336044469, 352.39460710117095])

In [23]:
Conv_backward_v7(input, weights, bias, gradient)

(Float32[2.4740026 2.673792 … 2.269287 1.7888533; 3.1967802 6.0427833 … 5.377192 2.864847; … ; 2.7728946 5.810988 … 6.5980916 2.5808516; 1.2643493 3.2989457 … 3.5650446 1.032854;;;], Float32[167.64406 169.15935 167.00087; 167.7004 163.9887 163.67015; 162.34265 163.48323 165.82878;;;; 163.57365 159.92496 164.4841; 161.50905 160.8499 161.48358; 161.07101 162.28336 162.46986;;;; 167.42563 168.78995 171.7227; 169.2116 166.07843 168.47705; 163.76366 163.80412 171.1175;;;; 176.75003 176.45345 177.18243; 170.60622 175.03868 177.14159; 171.02505 176.13356 177.88799;;;; 163.75742 166.89165 163.69727; 166.04155 163.56067 163.9592; 161.5332 162.68593 158.96265;;;; 171.92052 174.48372 173.80415; 171.91309 176.1706 172.15506; 170.91565 172.85696 175.36739], [338.2929296134087, 328.8824437877334, 341.81983651236527, 355.49778988061513, 333.1912336044469, 352.39460710117095])